CREATING A INTRACTIVE MULTIVARIATE DATA-VISULIZATION OF PUBLICATIONS DOMAIN

In [1]:
import numpy as np
import pandas as pd

import plotly
import plotly.express as px
import plotly.graph_objects as go

import jupyter_dash as jd
from jupyter_dash import JupyterDash

import dash
from dash import dcc
from dash import html
from dash import callback_context
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
from dash.dependencies import Output, Input, State, ALL, ALLSMALLER, MATCH

from urllib.parse import parse_qs, unquote
pd.options.display.max_columns = None

In [2]:
cutm=pd.read_excel('CUTM.xlsx')
cutm.head(5)

,Sl_No,Campus,School,Department,Name of Claimant of paper,Name of all Author(s) in order as indicated in the journal,Title of the Article,Name_of_the_Journal,ISSN,Volume,Issue,Page No.,Month,Year,Indexing(SCI/WoS/Scopus/UGC/NAAS/ICI/PubMed),Link to the recognition in UGC enlistment of the Journal,Is_Branch
0,1,Bhubaneswar,SoAS,Chemistry,Biswal Susanta Kumar,Biswal Susanta Kumar,Effect of formulation of variables on vitro re...,International Journal of Applied Chemistry,0973-9734,11,4,515-525,December,2015,Scopus,https://www.scopus.com/sourceid/19700173002,True
1,2,Bhubaneswar,SoAS,Chemistry,Biswal Susanta Kumar,Biswal Susanta Kumar,Dendritic Polymers for drug delivery applications,International Journal of Pharmaceutical Resear...,2277-3657,4,4,18-27,December,2015,WoS,https://mjl.clarivate.com:/search-results?issn...,True
2,3,Bhubaneswar,SoAS,Chemistry,Biswal Susanta Kumar,Biswal Susanta Kumar,Estimation of filtration potential of an effec...,Current World Environment,2320-8031,10,2,656-662,August,2015,Scopus,https://www.scopus.com/sourceid/21100854215,True
3,4,Bhubaneswar,SoAS,Physics,Subrata Sarangi,"Suchismita Mohanty, Subrata Sarangi, Gouri San...",Effect of Graphene/Montmorillonite on Chitosan...,International Journal of Physics and Research,2319-4499,5,4,29-38,August,2015,UGC,http://www.tjprc.org/journals/ugc-approved-jou...,True
4,5,Bhubaneswar,SoET,Computer Science and Engineering,P. Annan Naidu,P. Annan Naidu,Major Research Challenges in Data Mining,International Journal of Trend in Research and...,2394-9333,2,4,2021-05-09 00:00:00,August,2015,UGC,http://www.ijtrd.com/,True


In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

branches = cutm[cutm['Is_Branch']]['Campus'].drop_duplicates().sort_values().tolist()

main_layout = html.Div([
    html.Div([
    dbc.NavbarSimple([
            dbc.DropdownMenu([
                dbc.DropdownMenuItem(campus, href=campus) for campus in branches
            ], label='Select Campus'),
    ], brand='Home',brand_href='/', light=True),
    dbc.Row([
        dbc.Col(lg=1, md=1, sm=1),
        dbc.Col([
            
            dcc.Location(id='location'),
            html.Div(id='main_content')
        ], lg=10),
    ])
], style={'backgroundColor': '#A3E3FF'})
])


cutm_dashboard = html.Div([
    html.Br(),
        html.H1(id='branch_heading'),
        dbc.Row([
            dbc.Col(dcc.Graph(id='branch_page_graph'))
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Label('Select indicator:'),
                dcc.Dropdown(id='branch_page_indicator_dropdown',
                             placeholder='Choose an indicator',
                             value='Issue',
                             options=[{'label': indicator, 'value': indicator}
                                     for indicator in cutm.columns[3:16]]),                
            ]),
            dbc.Col([
                dbc.Label('Select Campus:'),
                dcc.Dropdown(id='branch_page_dropdown',
                             placeholder='Select one or more branch to compare',
                             multi=True,
                             options=[{'label': b, 'value': b}
                                       for b in branches]),
            ])
        ]),
        html.Br(), html.Br(),
        html.Div(id='branch_table')
])

indicators_dashboard = html.Div([
    html.H1("CUTM PUBLICATIONS OF 2020 ")
])

app.validation_layout = html.Div([
    main_layout,
    indicators_dashboard,
    cutm_dashboard,
])

app.layout = main_layout



@app.callback(Output('main_content', 'children'),
              Input('location', 'pathname'))
def display_content(pathname):
    if unquote(pathname[1:]) in branches:
        return cutm_dashboard
    else:
        return indicators_dashboard

@app.callback(Output('branch_page_dropdown', 'value'),
              Input('location', 'pathname'))
def set_dropdown_values(pathname):
    if unquote(pathname[1:]) in branches:
        branch = unquote(pathname[1:])
        return [branch]

@app.callback(Output('branch_heading', 'children'),
              Output('branch_page_graph', 'figure'),
              Output('branch_table', 'children'),
              Input('location', 'pathname'),
              Input('branch_page_dropdown', 'value'),
              Input('branch_page_indicator_dropdown', 'value'))
def plot_branch_charts(pathname, branches, indicator):
    if (not branches) or (not indicator):
        raise PreventUpdate
    if unquote(pathname[1:]) in branches:
        branch = unquote(pathname[1:])
    df = cutm[cutm['Is_Branch'] & cutm['Campus'].isin(branches)]
    fig = px.line(df,
                  x='Name_of_the_Journal',
                  y=indicator,
                  title='<b>' + indicator + '</b><br>' + ', '.join(branches),
                  color='Campus')
    fig.layout.paper_bgcolor = '#E5ECF6'
    table = cutm[cutm['Issue'] == branches[0]].T.reset_index()
    if table.shape[1] == 2:
        table.columns = [branches[0] + ' Info', '']
        table = dbc.Table.from_dataframe(table)
    else:
        table = html.Div()
    return branch + ' Publication Data', fig, table

app.run_server(port=8989)

Dash app running on http://127.0.0.1:8989/


CREATING A INTRACTIVE MULTIVARIATE DATA-VISUALIZATION OF STUDENT REGISTRATION DOMAIN

In [4]:
import numpy as np
import pandas as pd

import plotly
import plotly.express as px
import plotly.graph_objects as go

import jupyter_dash as jd
from jupyter_dash import JupyterDash

import dash
from dash import dcc
from dash import html
from dash import callback_context
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
from dash.dependencies import Output, Input, State, ALL, ALLSMALLER, MATCH

from urllib.parse import parse_qs, unquote
pd.options.display.max_columns = None

In [5]:
cutm = pd.read_excel('Student Registration.xlsx')
cutm.head(5)

,SLNO.,rollno,name,email,sub_code,sub_name,School,Campus,sub_credit,lec_type,Is_Branch
0,1,190804130011,ANEMU SOWJANYA,190804130011@cutm.ac.in,ASAC2202,"MANURES, FERTILIZERS & SOIL FERTILITY MANAGEMENT",M.S.Swaminathan School of Agriculture,Paralakhemundi,2,PP,True
1,2,190804130050,JAGANA VAMSIKRISHNA,190804130050@cutm.ac.in,ASAC2202,"MANURES, FERTILIZERS & SOIL FERTILITY MANAGEMENT",M.S.Swaminathan School of Agriculture,Paralakhemundi,2,PP,True
2,3,190804130112,LAVETI PURNIMA,190804130112@cutm.ac.in,ASAC2202,"MANURES, FERTILIZERS & SOIL FERTILITY MANAGEMENT",M.S.Swaminathan School of Agriculture,Paralakhemundi,2,PP,True
3,4,190804130162,PONNANA HARIPRIYA,190804130162@cutm.ac.in,ASAC2202,"MANURES, FERTILIZERS & SOIL FERTILITY MANAGEMENT",M.S.Swaminathan School of Agriculture,Paralakhemundi,2,PP,True
4,5,190804130164,CHITTABATTALA LIKHITHA,190804130164@cutm.ac.in,ASAC2202,"MANURES, FERTILIZERS & SOIL FERTILITY MANAGEMENT",M.S.Swaminathan School of Agriculture,Paralakhemundi,2,PP,True


In [6]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

branches = cutm[cutm['Is_Branch']]['Campus'].drop_duplicates().sort_values().tolist()

main_layout = html.Div([
    html.Div([
    dbc.NavbarSimple([
            dbc.DropdownMenu([
                dbc.DropdownMenuItem(campus, href=campus) for campus in branches
            ], label='Select Branches'),
    ], brand='Home',brand_href='/', light=True),
    dbc.Row([
        dbc.Col(lg=1, md=1, sm=1),
        dbc.Col([
            
            dcc.Location(id='location'),
            html.Div(id='main_content')
        ], lg=10),
    ])
], style={'backgroundColor': '#FFFF00',
         'text': '#000000'})
])

cutm_dashboard = html.Div([
    html.Br(),
        html.H1(id='student_heading'),
        dbc.Row([
            dbc.Col(dcc.Graph(id='student_page_graph'))
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Label('Select indicator:'),
                dcc.Dropdown(id='student_page_indicator_dropdown',
                             placeholder='Choose an indicator',
                             value='rollno',
                             options=[{'label': indicator, 'value': indicator}
                                     for indicator in cutm.columns[1:10]]),                
            ]),
            dbc.Col([
                dbc.Label('Select Branches:'),
                dcc.Dropdown(id='student_page_dropdown',
                             placeholder='Select one or more branch to compare',
                             multi=True,
                             options=[{'label': b, 'value': b}
                                       for b in branches]),
            ])
        ]),
        html.Br(), html.Br(),
        html.Div(id='student_table')
])

indicators_dashboard = html.Div([
    html.H1("CUTM STUDENT REGISTRATION")
])

app.validation_layout = html.Div([
    main_layout,
    indicators_dashboard,
    cutm_dashboard,
])

app.layout = main_layout

@app.callback(Output('main_content', 'children'),
              Input('location', 'pathname'))
def display_content(pathname):
    if unquote(pathname[1:]) in branches:
        return cutm_dashboard
    else:
        return indicators_dashboard

@app.callback(Output('student_page_dropdown', 'value'),
              Input('location', 'pathname'))
def set_dropdown_values(pathname):
    if unquote(pathname[1:]) in branches:
        branch = unquote(pathname[1:])
        return [branch]

@app.callback(Output('student_heading', 'children'),
              Output('student_page_graph', 'figure'),
              Output('student_table', 'children'),
              Input('location', 'pathname'),
              Input('student_page_dropdown', 'value'),
              Input('student_page_indicator_dropdown', 'value'))
def plot_branch_charts(pathname, branches, indicator):
    if (not branches) or (not indicator):
        raise PreventUpdate
    if unquote(pathname[1:]) in branches:
        branch = unquote(pathname[1:])
    df = cutm[cutm['Is_Branch'] & cutm['Campus'].isin(branches)]
    fig = px.line(df,
                  x='sub_code',
                  y=indicator,
                  title='<b>' + indicator + '</b><br>' + ', '.join(branches),
                  color='Campus')
    fig.layout.paper_bgcolor = '#B8C1FD'
    table = cutm[cutm['name'] == branches[0]].T.reset_index()
    if table.shape[1] == 2:
        table.columns = [branches[0] + ' Info', '']
        table = dbc.Table.from_dataframe(table)
    else:
        table = html.Div()
    return branch + ' Registration Data', fig, table


app.run_server(port=4895)

Dash app running on http://127.0.0.1:4895/
